In [171]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# import streamlit as st

In [172]:
df_precos = pd.read_excel('Case Dados - Analise de Ações.xlsx', sheet_name='PREÇOS')
df_setor = pd.read_excel('Case Dados - Analise de Ações.xlsx', sheet_name='SETOR')
df_macro = pd.read_excel('Case Dados - Analise de Ações.xlsx', sheet_name='MACRO')

In [173]:
df_setor.head(10)


,SETOR ECONÔMICO,SUBSETOR,SEGMENTO,PAPEL
0,Bens Industriais,Máquinas e Equipamentos,Motores e Compressores e Outros,WEGE3
1,Bens Industriais,Material de Transporte,Material Aeronáutico e de Defesa,EMBR3
2,Bens Industriais,Material de Transporte,Material Rodoviário,POMO4
3,Bens Industriais,Transporte,Exploração de Rodovias,CCRO3
4,Bens Industriais,Transporte,Exploração de Rodovias,ECOR3
5,Bens Industriais,Transporte,Serviços de Apoio e Armazenagem,STBP3
6,Bens Industriais,Transporte,Transporte Aéreo,AZUL4
7,Bens Industriais,Transporte,Transporte Aéreo,GOLL4
8,Bens Industriais,Transporte,Transporte Ferroviário,RAIL3
9,Comunicações,Telecomunicações,Telecomunicações,TIMS3


In [174]:
df_precos.head(10)

,DATA,PAPEL,NOME,PREÇO (R$)
0,2023-01-31,ABEV3,AMBEV S/A,13.66
1,2023-02-28,ABEV3,AMBEV S/A,13.39
2,2023-03-31,ABEV3,AMBEV S/A,14.33
3,2023-04-30,ABEV3,AMBEV S/A,14.16
4,2023-05-31,ABEV3,AMBEV S/A,14.41
5,2023-06-30,ABEV3,AMBEV S/A,15.41
6,2023-01-31,AESB3,AES BRASIL,10.10
7,2023-02-28,AESB3,AES BRASIL,9.92
8,2023-03-31,AESB3,AES BRASIL,9.94
9,2023-04-30,AESB3,AES BRASIL,10.68


In [175]:
df_macro.head(10)

,DATA,IPCA,SELIC,USD
0,2023-01-31,0.0053,0.0112,5.0987
1,2023-02-28,0.0084,0.0092,5.2072
2,2023-03-31,0.0071,0.0117,5.0798
3,2023-04-30,0.0061,0.0092,5.0001
4,2023-05-31,0.0023,0.0112,5.0953
5,2023-06-30,-0.0008,0.0107,4.8186
6,2023-07-31,0.0012,0.0107,4.7409
7,2023-08-31,0.0023,0.0114,4.9213
8,2023-09-30,0.0026,0.0097,5.0070
9,2023-10-31,0.0024,0.0100,5.0569


In [176]:
df_precos.info()
df_setor.info()
df_macro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528 entries, 0 to 527
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   DATA        528 non-null    datetime64[ns]
 1   PAPEL       528 non-null    object        
 2   NOME        528 non-null    object        
 3   PREÇO (R$)  528 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 16.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   SETOR ECONÔMICO  88 non-null     object
 1   SUBSETOR         88 non-null     object
 2   SEGMENTO         88 non-null     object
 3   PAPEL            88 non-null     object
dtypes: object(4)
memory usage: 2.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column  Non-Null 

In [177]:
df_macro[df_macro.isna().any(axis=1)]
df_precos[df_precos.isna().any(axis=1)]
df_setor[df_setor.isna().any(axis=1)]
df_precos[(df_precos['NOME'].isna() | df_precos['PAPEL'].isna() | df_precos['PREÇO (R$)'].isna() | df_precos['DATA'].isna())]

,DATA,PAPEL,NOME,PREÇO (R$)


In [178]:
df_macro.isna().sum()

DATA     0
IPCA     0
SELIC    0
USD      0
dtype: int64

In [179]:
df_macro.isna().sum() / len(df_macro)

DATA     0.0
IPCA     0.0
SELIC    0.0
USD      0.0
dtype: float64

# Transformando/ juntando os dados

In [180]:
df_precos_setor = pd.merge(df_precos, df_setor, how='outer', on='PAPEL')
df_precos_setor_macro12 = pd.merge(df_precos_setor, df_macro, how='right', on='DATA').fillna('macro')
df_precos_setor_macro6 = pd.merge(df_precos_setor, df_macro, how='left', on='DATA').fillna('macro')

# Criando colunas com os retornos

In [181]:

df_precos_setor_macro12['RETORNOS_SIMPLES'] = df_precos_setor_macro12.groupby(df_precos_setor_macro12.index)['PREÇO (R$)'].pct_change()
df_precos_setor_macro12['RETORNOS_ACUMULADOS'] = df_precos_setor_macro12.groupby(df_precos_setor_macro12.index)['RETORNOS_SIMPLES'].transform(lambda x : ((x + 1).cumprod() - 1))


In [182]:
df_precos_setor_macro6.set_index('PAPEL', drop=True, inplace=True)

In [183]:
df_precos_setor_macro6['RETORNOS_SIMPLES'] = df_precos_setor_macro6.groupby(df_precos_setor_macro6.index)['PREÇO (R$)'].pct_change()
df_precos_setor_macro6['RETORNOS_LOGARITIMOS'] = np.log(df_precos_setor_macro6.groupby(df_precos_setor_macro6.index)['PREÇO (R$)'].pct_change() + 1)

df_precos_setor_macro6['RETORNOS_ACUMULADOS'] = df_precos_setor_macro6.groupby(df_precos_setor_macro6.index)['RETORNOS_SIMPLES'].transform(lambda x : ((x + 1).cumprod() - 1))
df_precos_setor_macro6.head(15)


,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,RETORNOS_LOGARITIMOS,RETORNOS_ACUMULADOS
PAPEL,,,,,,,,,,,,
ABEV3,2023-01-31,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,NaN,NaN,NaN
ABEV3,2023-02-28,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,-0.019766,-0.019964,-0.019766
ABEV3,2023-03-31,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,0.070202,0.067847,0.049048
ABEV3,2023-04-30,AMBEV S/A,14.16,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0061,0.0092,5.0001,-0.011863,-0.011934,0.036603
ABEV3,2023-05-31,AMBEV S/A,14.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0023,0.0112,5.0953,0.017655,0.017501,0.054905
ABEV3,2023-06-30,AMBEV S/A,15.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,-0.0008,0.0107,4.8186,0.069396,0.067094,0.128111
AESB3,2023-01-31,AES BRASIL,10.10,Utilidade Pública,Energia Elétrica,Energia Elétrica,0.0053,0.0112,5.0987,NaN,NaN,NaN
AESB3,2023-02-28,AES BRASIL,9.92,Utilidade Pública,Energia Elétrica,Energia Elétrica,0.0084,0.0092,5.2072,-0.017822,-0.017983,-0.017822
AESB3,2023-03-31,AES BRASIL,9.94,Utilidade Pública,Energia Elétrica,Energia Elétrica,0.0071,0.0117,5.0798,0.002016,0.002014,-0.015842


# pivotando a tabela simples para plotar fgrafico de performance

In [184]:
df_precos_pivotada = df_precos.pivot_table(index='DATA', columns='PAPEL', values='PREÇO (R$)')
df_precos_pivotada

PAPEL,ABEV3,AESB3,ALPA4,ALSO3,ANIM3,ASAI3,AZUL4,B3SA3,BBSE3,BEEF3,...,TIMS3,TOTS3,TRPL4,UGPA3,USIM5,VALE3,VAMO3,VIVT3,WEGE3,YDUQ3
DATA,,,,,,,,,,,,,,,,,,,,,
2023-01-31,13.66,10.10,13.61,18.01,4.42,19.65,11.85,12.96,37.78,13.99,...,11.79,30.07,22.87,13.19,8.53,94.51,14.62,41.82,38.24,10.33
2023-02-28,13.39,9.92,9.43,17.08,3.19,18.14,7.13,10.55,34.21,11.33,...,12.26,27.29,21.76,13.18,6.83,85.32,13.28,38.94,39.16,7.05
2023-03-31,14.33,9.94,8.15,17.60,2.51,15.54,12.03,10.35,32.51,10.54,...,12.58,28.16,21.67,13.96,7.29,80.29,13.35,38.66,40.55,7.05
2023-04-30,14.16,10.68,7.41,18.64,2.31,12.29,10.86,11.68,34.31,8.55,...,14.00,25.66,23.05,14.44,7.20,72.40,12.22,41.12,41.12,8.12
2023-05-31,14.41,12.18,10.44,21.67,2.99,10.75,16.85,13.32,30.93,10.22,...,13.96,28.76,24.40,16.93,7.02,63.81,12.87,40.00,37.61,14.07
2023-06-30,15.41,12.25,9.30,24.65,4.19,13.75,21.86,14.61,30.77,10.63,...,14.62,29.98,25.36,18.89,7.07,64.22,12.11,43.27,37.75,19.82


# normalizando os preços

In [185]:
df_precos_pivotada_normalizada = df_precos_pivotada / df_precos_pivotada.iloc[0]
df_precos_pivotada_normalizada

PAPEL,ABEV3,AESB3,ALPA4,ALSO3,ANIM3,ASAI3,AZUL4,B3SA3,BBSE3,BEEF3,...,TIMS3,TOTS3,TRPL4,UGPA3,USIM5,VALE3,VAMO3,VIVT3,WEGE3,YDUQ3
DATA,,,,,,,,,,,,,,,,,,,,,
2023-01-31,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2023-02-28,0.980234,0.982178,0.692873,0.948362,0.721719,0.923155,0.601688,0.814043,0.905506,0.809864,...,1.039864,0.907549,0.951465,0.999242,0.800703,0.902762,0.908345,0.931133,1.024059,0.682478
2023-03-31,1.049048,0.984158,0.598824,0.977235,0.567873,0.790840,1.015190,0.798611,0.860508,0.753395,...,1.067006,0.936482,0.947530,1.058378,0.854631,0.849540,0.913133,0.924438,1.060408,0.682478
2023-04-30,1.036603,1.057426,0.544453,1.034981,0.522624,0.625445,0.916456,0.901235,0.908152,0.611151,...,1.187447,0.853342,1.007871,1.094769,0.844080,0.766057,0.835841,0.983262,1.075314,0.786060
2023-05-31,1.054905,1.205941,0.767083,1.203220,0.676471,0.547074,1.421941,1.027778,0.818687,0.730522,...,1.184054,0.956435,1.066900,1.283548,0.822978,0.675167,0.880301,0.956480,0.983525,1.362052
2023-06-30,1.128111,1.212871,0.683321,1.368684,0.947964,0.699746,1.844726,1.127315,0.814452,0.759828,...,1.240034,0.997007,1.108876,1.432146,0.828839,0.679505,0.828317,1.034672,0.987186,1.918683


#### ----

### com o PX

#### ----

# TOP Maiores retornos / menores retornos

In [186]:
df_precos_setor_macro6.head(10)

,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,RETORNOS_LOGARITIMOS,RETORNOS_ACUMULADOS
PAPEL,,,,,,,,,,,,
ABEV3,2023-01-31,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,NaN,NaN,NaN
ABEV3,2023-02-28,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,-0.019766,-0.019964,-0.019766
ABEV3,2023-03-31,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,0.070202,0.067847,0.049048
ABEV3,2023-04-30,AMBEV S/A,14.16,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0061,0.0092,5.0001,-0.011863,-0.011934,0.036603
ABEV3,2023-05-31,AMBEV S/A,14.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0023,0.0112,5.0953,0.017655,0.017501,0.054905
ABEV3,2023-06-30,AMBEV S/A,15.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,-0.0008,0.0107,4.8186,0.069396,0.067094,0.128111
AESB3,2023-01-31,AES BRASIL,10.10,Utilidade Pública,Energia Elétrica,Energia Elétrica,0.0053,0.0112,5.0987,NaN,NaN,NaN
AESB3,2023-02-28,AES BRASIL,9.92,Utilidade Pública,Energia Elétrica,Energia Elétrica,0.0084,0.0092,5.2072,-0.017822,-0.017983,-0.017822
AESB3,2023-03-31,AES BRASIL,9.94,Utilidade Pública,Energia Elétrica,Energia Elétrica,0.0071,0.0117,5.0798,0.002016,0.002014,-0.015842


In [187]:
primeira_cotacao = df_precos_setor_macro6.groupby(df_precos_setor_macro6.index)['PREÇO (R$)'].first()
primeira_cotacao.head(6)

PAPEL
ABEV3    13.66
AESB3    10.10
ALPA4    13.61
ALSO3    18.01
ANIM3     4.42
ASAI3    19.65
Name: PREÇO (R$), dtype: float64

In [188]:
ultima_cotacao = df_precos_setor_macro6.groupby(df_precos_setor_macro6.index)['PREÇO (R$)'].last()
ultima_cotacao.head(6)

PAPEL
ABEV3    15.41
AESB3    12.25
ALPA4     9.30
ALSO3    24.65
ANIM3     4.19
ASAI3    13.75
Name: PREÇO (R$), dtype: float64

In [189]:
top_valorizadas = ((ultima_cotacao - primeira_cotacao) / primeira_cotacao).sort_values(ascending=False)
# df_precos_setor_macro6[['RETORNOS_ACUMULADOS']].groupby(df_precos_setor_macro6.index).last()

In [190]:
top_desvalorizadas = ((ultima_cotacao - primeira_cotacao) / primeira_cotacao).sort_values(ascending=True)

### Calculando Rentabilidade BRUTA em R$

In [191]:
ganho_reais = ultima_cotacao - primeira_cotacao
ganho_reais

PAPEL
ABEV3     1.75
AESB3     2.15
ALPA4    -4.31
ALSO3     6.64
ANIM3    -0.23
         ...  
VALE3   -30.29
VAMO3    -2.51
VIVT3     1.45
WEGE3    -0.49
YDUQ3     9.49
Name: PREÇO (R$), Length: 88, dtype: float64

# Calculando o risco / volatilidade da ação no periodo analisado

### calculo do desvio padrao mensal

In [192]:
volatilidade = df_precos_setor_macro6['RETORNOS_SIMPLES'].dropna().groupby(df_precos_setor_macro6.dropna().index).std(ddof=1) #ddof=1 amostral e populacional = 0
volatilidade
# maiores_volatilidades = volatilidade

PAPEL
ABEV3    0.043101
AESB3    0.065634
ALPA4    0.268876
ALSO3    0.085987
ANIM3    0.305717
           ...   
VALE3    0.049535
VAMO3    0.062553
VIVT3    0.063075
WEGE3    0.048279
YDUQ3    0.399190
Name: RETORNOS_SIMPLES, Length: 88, dtype: float64

In [193]:
df_precos_pivotada['IPCA'] = df_precos_setor_macro6['IPCA'][0:6].values
df_precos_pivotada['SELIC'] = df_precos_setor_macro6['SELIC'][0:6].values
df_precos_pivotada['SELIC_acumulada'] = df_precos_setor_macro6['SELIC'][0:6].values.cumsum()
df_precos_pivotada['IPCA_acumulado'] = df_precos_setor_macro6['IPCA'][0:6].values.cumsum()

df_precos_pivotada['USD'] = df_precos_setor_macro6['USD'][0:6].values

df_precos_pivotada

PAPEL,ABEV3,AESB3,ALPA4,ALSO3,ANIM3,ASAI3,AZUL4,B3SA3,BBSE3,BEEF3,...,VALE3,VAMO3,VIVT3,WEGE3,YDUQ3,IPCA,SELIC,SELIC_acumulada,IPCA_acumulado,USD
DATA,,,,,,,,,,,,,,,,,,,,,
2023-01-31,13.66,10.10,13.61,18.01,4.42,19.65,11.85,12.96,37.78,13.99,...,94.51,14.62,41.82,38.24,10.33,0.0053,0.0112,0.0112,0.0053,5.0987
2023-02-28,13.39,9.92,9.43,17.08,3.19,18.14,7.13,10.55,34.21,11.33,...,85.32,13.28,38.94,39.16,7.05,0.0084,0.0092,0.0204,0.0137,5.2072
2023-03-31,14.33,9.94,8.15,17.60,2.51,15.54,12.03,10.35,32.51,10.54,...,80.29,13.35,38.66,40.55,7.05,0.0071,0.0117,0.0321,0.0208,5.0798
2023-04-30,14.16,10.68,7.41,18.64,2.31,12.29,10.86,11.68,34.31,8.55,...,72.40,12.22,41.12,41.12,8.12,0.0061,0.0092,0.0413,0.0269,5.0001
2023-05-31,14.41,12.18,10.44,21.67,2.99,10.75,16.85,13.32,30.93,10.22,...,63.81,12.87,40.00,37.61,14.07,0.0023,0.0112,0.0525,0.0292,5.0953
2023-06-30,15.41,12.25,9.30,24.65,4.19,13.75,21.86,14.61,30.77,10.63,...,64.22,12.11,43.27,37.75,19.82,-0.0008,0.0107,0.0632,0.0284,4.8186


In [194]:
df_precos_setor_macro6['IPCA'][0:6].values

array([ 0.0053,  0.0084,  0.0071,  0.0061,  0.0023, -0.0008])

# Calculando o indice de sharpe
- Supondo que retorno livre de risco +- 4,5%

In [195]:
risco_free = 0.045
sharpe = (top_valorizadas.sort_index() - risco_free) / volatilidade.sort_index()
sharpe.head(40)

PAPEL
ABEV3     1.928270
AESB3     2.557681
ALPA4    -1.345150
ALSO3     3.764327
ANIM3    -0.317405
ASAI3    -1.791367
AZUL4     1.770101
B3SA3     0.599655
BBSE3    -3.559116
BEEF3    -1.740929
BPAC11    3.333036
BRFS3     0.406214
BRKM5     1.011555
CBAV3    -4.854580
CCRO3     1.767746
CEAB3     2.092709
CIEL3    -1.321789
CMIG4     0.874571
CMIN3    -4.456727
COGN3     1.478318
CPFE3    -0.120056
CPLE6     0.194583
CRFB3    -2.531472
CSAN3     0.229391
CSMG3     5.515623
CSNA3    -7.507624
CVCB3    -1.251618
CYRE3     2.690102
DIRR3     2.528185
ECOR3     2.680841
EGIE3     3.483511
EMBR3     0.698618
ENEV3    -0.307875
ENGI11    1.790497
EQTL3     1.208495
EZTC3     1.426052
FLRY3    -0.141223
GGBR4    -3.815287
GMAT3     0.614043
GOLL4     2.108912
dtype: float64

# Max Drowndown calculando:

In [196]:
precos_max_dd = df_precos_setor_macro6.groupby(df_precos_setor_macro6.index)['PREÇO (R$)'].cummax()
# precos = df_precos_setor_macro6.groupby(df_precos_setor_macro6)[['PREÇO (R$)']]

precos_max_dd.head(6)

PAPEL
ABEV3    13.66
ABEV3    13.66
ABEV3    14.33
ABEV3    14.33
ABEV3    14.41
ABEV3    15.41
Name: PREÇO (R$), dtype: float64

In [197]:
precos = df_precos_setor_macro6['PREÇO (R$)']
precos.head(6)

PAPEL
ABEV3    13.66
ABEV3    13.39
ABEV3    14.33
ABEV3    14.16
ABEV3    14.41
ABEV3    15.41
Name: PREÇO (R$), dtype: float64

In [198]:
drowdowns = ((precos_max_dd - precos) / precos_max_dd)
drowdowns.head(60)
drowdowns_neg = ((precos - precos_max_dd ) / precos_max_dd)
drowdowns_neg.head(60)

PAPEL
ABEV3    0.000000
ABEV3   -0.019766
ABEV3    0.000000
ABEV3   -0.011863
ABEV3    0.000000
ABEV3    0.000000
AESB3    0.000000
AESB3   -0.017822
AESB3   -0.015842
AESB3    0.000000
AESB3    0.000000
AESB3    0.000000
ALPA4    0.000000
ALPA4   -0.307127
ALPA4   -0.401176
ALPA4   -0.455547
ALPA4   -0.232917
ALPA4   -0.316679
ALSO3    0.000000
ALSO3   -0.051638
ALSO3   -0.022765
ALSO3    0.000000
ALSO3    0.000000
ALSO3    0.000000
ANIM3    0.000000
ANIM3   -0.278281
ANIM3   -0.432127
ANIM3   -0.477376
ANIM3   -0.323529
ANIM3   -0.052036
ASAI3    0.000000
ASAI3   -0.076845
ASAI3   -0.209160
ASAI3   -0.374555
ASAI3   -0.452926
ASAI3   -0.300254
AZUL4    0.000000
AZUL4   -0.398312
AZUL4    0.000000
AZUL4   -0.097257
AZUL4    0.000000
AZUL4    0.000000
B3SA3    0.000000
B3SA3   -0.185957
B3SA3   -0.201389
B3SA3   -0.098765
B3SA3    0.000000
B3SA3    0.000000
BBSE3    0.000000
BBSE3   -0.094494
BBSE3   -0.139492
BBSE3   -0.091848
BBSE3   -0.181313
BBSE3   -0.185548
BEEF3    0.000000
BEEF

### Maiore DD's

In [199]:
max_dd = drowdowns_neg.groupby(drowdowns.index).min()
max_dd.sort_values().head(40)


PAPEL
CBAV3   -0.603150
MLAS3   -0.591045
HAPV3   -0.491262
ANIM3   -0.477376
ALPA4   -0.455547
RECV3   -0.454402
ASAI3   -0.452926
CRFB3   -0.424316
QUAL3   -0.416799
AZUL4   -0.398312
CVCB3   -0.395556
BEEF3   -0.388849
CSNA3   -0.344324
RRRP3   -0.342628
RDOR3   -0.334169
VALE3   -0.324833
CEAB3   -0.322148
YDUQ3   -0.317522
PCAR3   -0.289093
NTCO3   -0.278068
GOLL4   -0.278068
GGBR4   -0.267868
LREN3   -0.267684
PRIO3   -0.259145
MGLU3   -0.252822
LWSA3   -0.252730
SOMA3   -0.227619
BRFS3   -0.226131
CMIN3   -0.217228
COGN3   -0.214286
B3SA3   -0.201389
USIM5   -0.199297
HYPE3   -0.197198
GMAT3   -0.194444
BBSE3   -0.185548
MRFG3   -0.172727
VAMO3   -0.171683
SBSP3   -0.171218
BRKM5   -0.170167
EZTC3   -0.168601
Name: PREÇO (R$), dtype: float64

In [200]:
lista_piores_dd = list(max_dd.sort_values().index)
lista_piores_dd

['CBAV3',
 'MLAS3',
 'HAPV3',
 'ANIM3',
 'ALPA4',
 'RECV3',
 'ASAI3',
 'CRFB3',
 'QUAL3',
 'AZUL4',
 'CVCB3',
 'BEEF3',
 'CSNA3',
 'RRRP3',
 'RDOR3',
 'VALE3',
 'CEAB3',
 'YDUQ3',
 'PCAR3',
 'NTCO3',
 'GOLL4',
 'GGBR4',
 'LREN3',
 'PRIO3',
 'MGLU3',
 'LWSA3',
 'SOMA3',
 'BRFS3',
 'CMIN3',
 'COGN3',
 'B3SA3',
 'USIM5',
 'HYPE3',
 'GMAT3',
 'BBSE3',
 'MRFG3',
 'VAMO3',
 'SBSP3',
 'BRKM5',
 'EZTC3',
 'SUZB3',
 'JBSS3',
 'MRVE3',
 'CIEL3',
 'PETZ3',
 'TOTS3',
 'RAIZ4',
 'ODPV3',
 'MOVI3',
 'ENEV3',
 'PSSA3',
 'EMBR3',
 'CPLE6',
 'CSAN3',
 'PETR4',
 'RENT3',
 'ECOR3',
 'EQTL3',
 'BPAC11',
 'STBP3',
 'CPFE3',
 'RADL3',
 'CMIG4',
 'WEGE3',
 'ENGI11',
 'FLRY3',
 'SIMH3',
 'VIVT3',
 'SANB11',
 'CYRE3',
 'KLBN11',
 'CCRO3',
 'TAEE11',
 'SMTO3',
 'TRPL4',
 'ALSO3',
 'ITSA4',
 'DIRR3',
 'CSMG3',
 'ITUB4',
 'RAIL3',
 'MULT3',
 'ABEV3',
 'AESB3',
 'TIMS3',
 'EGIE3',
 'UGPA3',
 'POMO4']

## calculando os valores brutos

In [201]:
drowdowns_bruto = precos_max_dd - precos
drowdowns_neg_bruto = precos - precos_max_dd

max_dd_bruto = drowdowns_neg_bruto.groupby(drowdowns_bruto.index).min()
max_dd_bruto.sort_values().head(40)

PAPEL
VALE3    -30.70
RRRP3    -15.36
RECV3    -14.35
PRIO3    -10.91
RDOR3    -10.66
SBSP3     -9.53
HYPE3     -9.15
ASAI3     -8.90
GGBR4     -8.77
SUZB3     -7.94
CBAV3     -7.66
BBSE3     -7.01
CRFB3     -6.98
CSNA3     -6.37
ALPA4     -6.20
PCAR3     -5.99
RENT3     -5.81
LREN3     -5.79
BEEF3     -5.44
AZUL4     -4.72
TOTS3     -4.41
NTCO3     -4.26
BRKM5     -3.97
ENGI11    -3.56
WEGE3     -3.51
YDUQ3     -3.28
JBSS3     -3.28
VIVT3     -3.16
CPFE3     -3.00
PSSA3     -2.95
QUAL3     -2.63
PETR4     -2.62
B3SA3     -2.61
EQTL3     -2.59
HAPV3     -2.53
VAMO3     -2.51
EZTC3     -2.47
SOMA3     -2.39
EMBR3     -2.25
TAEE11    -2.23
Name: PREÇO (R$), dtype: float64

In [202]:
lista_piores_dd_bruto = list(max_dd_bruto.sort_values().index)
lista_piores_dd_bruto

['VALE3',
 'RRRP3',
 'RECV3',
 'PRIO3',
 'RDOR3',
 'SBSP3',
 'HYPE3',
 'ASAI3',
 'GGBR4',
 'SUZB3',
 'CBAV3',
 'BBSE3',
 'CRFB3',
 'CSNA3',
 'ALPA4',
 'PCAR3',
 'RENT3',
 'LREN3',
 'BEEF3',
 'AZUL4',
 'TOTS3',
 'NTCO3',
 'BRKM5',
 'ENGI11',
 'WEGE3',
 'YDUQ3',
 'JBSS3',
 'VIVT3',
 'CPFE3',
 'PSSA3',
 'QUAL3',
 'PETR4',
 'B3SA3',
 'EQTL3',
 'HAPV3',
 'VAMO3',
 'EZTC3',
 'SOMA3',
 'EMBR3',
 'TAEE11',
 'RADL3',
 'SANB11',
 'GOLL4',
 'ANIM3',
 'SMTO3',
 'MLAS3',
 'BPAC11',
 'BRFS3',
 'CVCB3',
 'CSAN3',
 'USIM5',
 'LWSA3',
 'ODPV3',
 'ENEV3',
 'KLBN11',
 'MRFG3',
 'GMAT3',
 'FLRY3',
 'TRPL4',
 'CMIN3',
 'MGLU3',
 'MRVE3',
 'CYRE3',
 'PETZ3',
 'CMIG4',
 'MOVI3',
 'CEAB3',
 'ALSO3',
 'CIEL3',
 'CPLE6',
 'STBP3',
 'CCRO3',
 'ITUB4',
 'SIMH3',
 'DIRR3',
 'MULT3',
 'COGN3',
 'ECOR3',
 'RAIZ4',
 'CSMG3',
 'RAIL3',
 'ITSA4',
 'ABEV3',
 'AESB3',
 'EGIE3',
 'TIMS3',
 'UGPA3',
 'POMO4']

In [203]:
df_media = df_precos_setor_macro6.groupby(df_precos_setor_macro6.index)['PREÇO (R$)'].mean().sort_values(ascending=False)
df_media.head(20)

PAPEL
VALE3     76.758333
RENT3     59.441667
SBSP3     52.211667
SUZB3     44.083333
ENGI11    42.860000
HYPE3     41.638333
EGIE3     41.291667
VIVT3     40.635000
WEGE3     39.071667
TAEE11    36.401667
PRIO3     35.501667
RRRP3     33.756667
BBSE3     33.418333
CPFE3     32.171667
SMTO3     30.158333
SANB11    28.363333
TOTS3     28.320000
EQTL3     28.008333
RDOR3     27.246667
GGBR4     26.813333
Name: PREÇO (R$), dtype: float64

# Juntando todos dados no DF

In [204]:
df_completo = df_precos_setor_macro6.copy(deep=True)
df_completo

,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,RETORNOS_LOGARITIMOS,RETORNOS_ACUMULADOS
PAPEL,,,,,,,,,,,,
ABEV3,2023-01-31,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,NaN,NaN,NaN
ABEV3,2023-02-28,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,-0.019766,-0.019964,-0.019766
ABEV3,2023-03-31,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,0.070202,0.067847,0.049048
ABEV3,2023-04-30,AMBEV S/A,14.16,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0061,0.0092,5.0001,-0.011863,-0.011934,0.036603
ABEV3,2023-05-31,AMBEV S/A,14.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0023,0.0112,5.0953,0.017655,0.017501,0.054905
...,...,...,...,...,...,...,...,...,...,...,...,...
YDUQ3,2023-02-28,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0084,0.0092,5.2072,-0.317522,-0.382025,-0.317522
YDUQ3,2023-03-31,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0071,0.0117,5.0798,0.000000,0.000000,-0.317522
YDUQ3,2023-04-30,YDUQS PART,8.12,Consumo Cíclico,Diversos,Serviços Educacionais,0.0061,0.0092,5.0001,0.151773,0.141303,-0.213940


#### JUNTANDO SHARPE

In [205]:
t1 = pd.merge(df_completo, sharpe.to_frame(name='SHARPE'), right_index=True,left_index=True, how='outer')
t1

,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,RETORNOS_LOGARITIMOS,RETORNOS_ACUMULADOS,SHARPE
PAPEL,,,,,,,,,,,,,
ABEV3,2023-01-31,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,NaN,NaN,NaN,1.928270
ABEV3,2023-02-28,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,-0.019766,-0.019964,-0.019766,1.928270
ABEV3,2023-03-31,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,0.070202,0.067847,0.049048,1.928270
ABEV3,2023-04-30,AMBEV S/A,14.16,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0061,0.0092,5.0001,-0.011863,-0.011934,0.036603,1.928270
ABEV3,2023-05-31,AMBEV S/A,14.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0023,0.0112,5.0953,0.017655,0.017501,0.054905,1.928270
...,...,...,...,...,...,...,...,...,...,...,...,...,...
YDUQ3,2023-02-28,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0084,0.0092,5.2072,-0.317522,-0.382025,-0.317522,2.188639
YDUQ3,2023-03-31,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0071,0.0117,5.0798,0.000000,0.000000,-0.317522,2.188639
YDUQ3,2023-04-30,YDUQS PART,8.12,Consumo Cíclico,Diversos,Serviços Educacionais,0.0061,0.0092,5.0001,0.151773,0.141303,-0.213940,2.188639


#### JUNTANDO MEDIA

In [206]:
t2 = pd.merge(t1, df_media.to_frame(name='MÉDIA'), right_index=True,left_index=True, how='outer')
t2

,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,RETORNOS_LOGARITIMOS,RETORNOS_ACUMULADOS,SHARPE,MÉDIA
PAPEL,,,,,,,,,,,,,,
ABEV3,2023-01-31,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,NaN,NaN,NaN,1.928270,14.226667
ABEV3,2023-02-28,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,-0.019766,-0.019964,-0.019766,1.928270,14.226667
ABEV3,2023-03-31,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,0.070202,0.067847,0.049048,1.928270,14.226667
ABEV3,2023-04-30,AMBEV S/A,14.16,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0061,0.0092,5.0001,-0.011863,-0.011934,0.036603,1.928270,14.226667
ABEV3,2023-05-31,AMBEV S/A,14.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0023,0.0112,5.0953,0.017655,0.017501,0.054905,1.928270,14.226667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YDUQ3,2023-02-28,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0084,0.0092,5.2072,-0.317522,-0.382025,-0.317522,2.188639,11.073333
YDUQ3,2023-03-31,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0071,0.0117,5.0798,0.000000,0.000000,-0.317522,2.188639,11.073333
YDUQ3,2023-04-30,YDUQS PART,8.12,Consumo Cíclico,Diversos,Serviços Educacionais,0.0061,0.0092,5.0001,0.151773,0.141303,-0.213940,2.188639,11.073333


#### JUNTANDO MAX DROW DOWN

In [207]:
t3 = pd.merge(t2,max_dd.to_frame(name='MÁXIMO DROW DOWN'), right_index=True,left_index=True, how='outer')
t3

,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,RETORNOS_LOGARITIMOS,RETORNOS_ACUMULADOS,SHARPE,MÉDIA,MÁXIMO DROW DOWN
PAPEL,,,,,,,,,,,,,,,
ABEV3,2023-01-31,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,NaN,NaN,NaN,1.928270,14.226667,-0.019766
ABEV3,2023-02-28,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,-0.019766,-0.019964,-0.019766,1.928270,14.226667,-0.019766
ABEV3,2023-03-31,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,0.070202,0.067847,0.049048,1.928270,14.226667,-0.019766
ABEV3,2023-04-30,AMBEV S/A,14.16,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0061,0.0092,5.0001,-0.011863,-0.011934,0.036603,1.928270,14.226667,-0.019766
ABEV3,2023-05-31,AMBEV S/A,14.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0023,0.0112,5.0953,0.017655,0.017501,0.054905,1.928270,14.226667,-0.019766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YDUQ3,2023-02-28,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0084,0.0092,5.2072,-0.317522,-0.382025,-0.317522,2.188639,11.073333,-0.317522
YDUQ3,2023-03-31,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0071,0.0117,5.0798,0.000000,0.000000,-0.317522,2.188639,11.073333,-0.317522
YDUQ3,2023-04-30,YDUQS PART,8.12,Consumo Cíclico,Diversos,Serviços Educacionais,0.0061,0.0092,5.0001,0.151773,0.141303,-0.213940,2.188639,11.073333,-0.317522


#### JUNTANDO MAXIMO DROW DOWN BRUTO

In [208]:
max_dd_bruto

PAPEL
ABEV3    -0.27
AESB3    -0.18
ALPA4    -6.20
ALSO3    -0.93
ANIM3    -2.11
         ...  
VALE3   -30.70
VAMO3    -2.51
VIVT3    -3.16
WEGE3    -3.51
YDUQ3    -3.28
Name: PREÇO (R$), Length: 88, dtype: float64

In [209]:
t4 = pd.merge(t3,max_dd_bruto.to_frame(name='MÁXIMO DROW DOWN bruto'), right_index=True,left_index=True, how='outer')
t4

,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,RETORNOS_LOGARITIMOS,RETORNOS_ACUMULADOS,SHARPE,MÉDIA,MÁXIMO DROW DOWN,MÁXIMO DROW DOWN bruto
PAPEL,,,,,,,,,,,,,,,,
ABEV3,2023-01-31,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,NaN,NaN,NaN,1.928270,14.226667,-0.019766,-0.27
ABEV3,2023-02-28,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,-0.019766,-0.019964,-0.019766,1.928270,14.226667,-0.019766,-0.27
ABEV3,2023-03-31,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,0.070202,0.067847,0.049048,1.928270,14.226667,-0.019766,-0.27
ABEV3,2023-04-30,AMBEV S/A,14.16,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0061,0.0092,5.0001,-0.011863,-0.011934,0.036603,1.928270,14.226667,-0.019766,-0.27
ABEV3,2023-05-31,AMBEV S/A,14.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0023,0.0112,5.0953,0.017655,0.017501,0.054905,1.928270,14.226667,-0.019766,-0.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YDUQ3,2023-02-28,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0084,0.0092,5.2072,-0.317522,-0.382025,-0.317522,2.188639,11.073333,-0.317522,-3.28
YDUQ3,2023-03-31,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0071,0.0117,5.0798,0.000000,0.000000,-0.317522,2.188639,11.073333,-0.317522,-3.28
YDUQ3,2023-04-30,YDUQS PART,8.12,Consumo Cíclico,Diversos,Serviços Educacionais,0.0061,0.0092,5.0001,0.151773,0.141303,-0.213940,2.188639,11.073333,-0.317522,-3.28


#### JUNTANDO VOLATILIDADE

In [210]:
t5 = pd.merge(t4,volatilidade.to_frame(name='VOLATILIDADE'), right_index=True,left_index=True, how='outer')
t5

,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,RETORNOS_LOGARITIMOS,RETORNOS_ACUMULADOS,SHARPE,MÉDIA,MÁXIMO DROW DOWN,MÁXIMO DROW DOWN bruto,VOLATILIDADE
PAPEL,,,,,,,,,,,,,,,,,
ABEV3,2023-01-31,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,NaN,NaN,NaN,1.928270,14.226667,-0.019766,-0.27,0.043101
ABEV3,2023-02-28,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,-0.019766,-0.019964,-0.019766,1.928270,14.226667,-0.019766,-0.27,0.043101
ABEV3,2023-03-31,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,0.070202,0.067847,0.049048,1.928270,14.226667,-0.019766,-0.27,0.043101
ABEV3,2023-04-30,AMBEV S/A,14.16,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0061,0.0092,5.0001,-0.011863,-0.011934,0.036603,1.928270,14.226667,-0.019766,-0.27,0.043101
ABEV3,2023-05-31,AMBEV S/A,14.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0023,0.0112,5.0953,0.017655,0.017501,0.054905,1.928270,14.226667,-0.019766,-0.27,0.043101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YDUQ3,2023-02-28,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0084,0.0092,5.2072,-0.317522,-0.382025,-0.317522,2.188639,11.073333,-0.317522,-3.28,0.399190
YDUQ3,2023-03-31,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0071,0.0117,5.0798,0.000000,0.000000,-0.317522,2.188639,11.073333,-0.317522,-3.28,0.399190
YDUQ3,2023-04-30,YDUQS PART,8.12,Consumo Cíclico,Diversos,Serviços Educacionais,0.0061,0.0092,5.0001,0.151773,0.141303,-0.213940,2.188639,11.073333,-0.317522,-3.28,0.399190


#### JUNTANDO RETORNO DO PERIODO

In [211]:
t6 = pd.merge(t5,top_valorizadas.to_frame(name='RENTABILIDADE'), right_index=True,left_index=True, how='outer')
t6

,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,RETORNOS_LOGARITIMOS,RETORNOS_ACUMULADOS,SHARPE,MÉDIA,MÁXIMO DROW DOWN,MÁXIMO DROW DOWN bruto,VOLATILIDADE,RENTABILIDADE
PAPEL,,,,,,,,,,,,,,,,,,
ABEV3,2023-01-31,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,NaN,NaN,NaN,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111
ABEV3,2023-02-28,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,-0.019766,-0.019964,-0.019766,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111
ABEV3,2023-03-31,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,0.070202,0.067847,0.049048,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111
ABEV3,2023-04-30,AMBEV S/A,14.16,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0061,0.0092,5.0001,-0.011863,-0.011934,0.036603,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111
ABEV3,2023-05-31,AMBEV S/A,14.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0023,0.0112,5.0953,0.017655,0.017501,0.054905,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YDUQ3,2023-02-28,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0084,0.0092,5.2072,-0.317522,-0.382025,-0.317522,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683
YDUQ3,2023-03-31,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0071,0.0117,5.0798,0.000000,0.000000,-0.317522,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683
YDUQ3,2023-04-30,YDUQS PART,8.12,Consumo Cíclico,Diversos,Serviços Educacionais,0.0061,0.0092,5.0001,0.151773,0.141303,-0.213940,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683


#### JUNTANDO RENTABILIDADE BRUTA DO PERIODO

In [212]:
t7 = pd.merge(t6,ganho_reais.to_frame(name='GANHO BRUTO R$'), right_index=True,left_index=True, how='outer')
t7

,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,RETORNOS_LOGARITIMOS,RETORNOS_ACUMULADOS,SHARPE,MÉDIA,MÁXIMO DROW DOWN,MÁXIMO DROW DOWN bruto,VOLATILIDADE,RENTABILIDADE,GANHO BRUTO R$
PAPEL,,,,,,,,,,,,,,,,,,,
ABEV3,2023-01-31,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,NaN,NaN,NaN,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,2023-02-28,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,-0.019766,-0.019964,-0.019766,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,2023-03-31,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,0.070202,0.067847,0.049048,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,2023-04-30,AMBEV S/A,14.16,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0061,0.0092,5.0001,-0.011863,-0.011934,0.036603,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,2023-05-31,AMBEV S/A,14.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0023,0.0112,5.0953,0.017655,0.017501,0.054905,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YDUQ3,2023-02-28,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0084,0.0092,5.2072,-0.317522,-0.382025,-0.317522,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49
YDUQ3,2023-03-31,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0071,0.0117,5.0798,0.000000,0.000000,-0.317522,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49
YDUQ3,2023-04-30,YDUQS PART,8.12,Consumo Cíclico,Diversos,Serviços Educacionais,0.0061,0.0092,5.0001,0.151773,0.141303,-0.213940,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49


In [285]:
df = t7.copy(deep=True)
df

,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,RETORNOS_LOGARITIMOS,RETORNOS_ACUMULADOS,SHARPE,MÉDIA,MÁXIMO DROW DOWN,MÁXIMO DROW DOWN bruto,VOLATILIDADE,RENTABILIDADE,GANHO BRUTO R$
PAPEL,,,,,,,,,,,,,,,,,,,
ABEV3,2023-01-31,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,NaN,NaN,NaN,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,2023-02-28,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,-0.019766,-0.019964,-0.019766,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,2023-03-31,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,0.070202,0.067847,0.049048,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,2023-04-30,AMBEV S/A,14.16,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0061,0.0092,5.0001,-0.011863,-0.011934,0.036603,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,2023-05-31,AMBEV S/A,14.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0023,0.0112,5.0953,0.017655,0.017501,0.054905,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YDUQ3,2023-02-28,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0084,0.0092,5.2072,-0.317522,-0.382025,-0.317522,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49
YDUQ3,2023-03-31,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0071,0.0117,5.0798,0.000000,0.000000,-0.317522,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49
YDUQ3,2023-04-30,YDUQS PART,8.12,Consumo Cíclico,Diversos,Serviços Educacionais,0.0061,0.0092,5.0001,0.151773,0.141303,-0.213940,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49


# Começando as analises e plot de graficos

In [214]:
from plotly import graph_objects as go
from plotly.subplots import make_subplots


In [216]:
df

,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,RETORNOS_LOGARITIMOS,RETORNOS_ACUMULADOS,SHARPE,MÉDIA,MÁXIMO DROW DOWN,MÁXIMO DROW DOWN bruto,VOLATILIDADE,RENTABILIDADE,GANHO BRUTO R$
PAPEL,,,,,,,,,,,,,,,,,,,
ABEV3,2023-01-31,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,NaN,NaN,NaN,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,2023-02-28,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,-0.019766,-0.019964,-0.019766,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,2023-03-31,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,0.070202,0.067847,0.049048,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,2023-04-30,AMBEV S/A,14.16,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0061,0.0092,5.0001,-0.011863,-0.011934,0.036603,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,2023-05-31,AMBEV S/A,14.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0023,0.0112,5.0953,0.017655,0.017501,0.054905,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YDUQ3,2023-02-28,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0084,0.0092,5.2072,-0.317522,-0.382025,-0.317522,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49
YDUQ3,2023-03-31,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0071,0.0117,5.0798,0.000000,0.000000,-0.317522,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49
YDUQ3,2023-04-30,YDUQS PART,8.12,Consumo Cíclico,Diversos,Serviços Educacionais,0.0061,0.0092,5.0001,0.151773,0.141303,-0.213940,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49


In [215]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 528 entries, ABEV3 to YDUQ3
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   DATA                    528 non-null    datetime64[ns]
 1   NOME                    528 non-null    object        
 2   PREÇO (R$)              528 non-null    float64       
 3   SETOR ECONÔMICO         528 non-null    object        
 4   SUBSETOR                528 non-null    object        
 5   SEGMENTO                528 non-null    object        
 6   IPCA                    528 non-null    float64       
 7   SELIC                   528 non-null    float64       
 8   USD                     528 non-null    float64       
 9   RETORNOS_SIMPLES        440 non-null    float64       
 10  RETORNOS_LOGARITIMOS    440 non-null    float64       
 11  RETORNOS_ACUMULADOS     440 non-null    float64       
 12  SHARPE                  528 non-null    float64  

# PLOT TODAS AÇÕES PARA VIZUALIZAÇÃO BASICA

In [217]:
import plotly.express as px

fig = px.line(df_precos_pivotada, x=df_precos_pivotada.index, y=df_precos_pivotada.columns, title='Evolução do Preço das Ações ao Longo do Tempo', markers='marker')#height=6000)

fig.update_layout(
    title='Evolução do Preço das Ações ao Longo do Tempo SEM NORMALIZAR',
    legend_title_text='TICKER',
    xaxis_title='Data',
    yaxis_title='Preço'
)

fig.update_xaxes(title_text='Data') # ALTERNATIVA
fig.update_yaxes(title_text='Preço') # ALTERNATIVA

fig.show()

# Plot de grafico acoes com retornos positivos NO FINAL DO PERIODO DE 6 MESES

In [218]:
ret_maiores_que_1 = df_precos_pivotada_normalizada.loc[:, df_precos_pivotada_normalizada.loc[df_precos_pivotada_normalizada.index[-1]] >= 1]
ret_maiores_que_1

figura = go.Figure()

for col in ret_maiores_que_1.columns:
    trace = go.Scatter(x=ret_maiores_que_1.index, y=ret_maiores_que_1[col], mode='lines+markers', name=col)
    figura.add_trace(trace)

figura.update_layout(title='Evolução do Preço das Ações ao Longo do Tempo')
figura.update_layout(legend_title_text='TICKER')

figura.update_layout(
    title='Evolução do Preço das Ações ao Longo do Tempo',
    legend_title_text='TICKER',
    xaxis_title='Data',
    yaxis_title='Preço',
    height=1000
)

figura.show()

# Plot de grafico acoes com retornos negativos NO FINAL DO PERIODO DE 6 MESES

In [219]:
ret_menores_que_1 = df_precos_pivotada_normalizada.loc[:, df_precos_pivotada_normalizada.loc[df_precos_pivotada_normalizada.index[-1]] <= 1]
ret_menores_que_1

figura = go.Figure()

for col in ret_menores_que_1.columns:
    trace = go.Scatter(x=ret_menores_que_1.index, y=ret_menores_que_1[col], mode='lines+markers', name=col)
    figura.add_trace(trace)

figura.update_layout(title='Evolução do Preço das Ações ao Longo do Tempo COM RENTABILIDADE RUIM')
figura.update_layout(legend_title_text='TICKER')

figura.update_layout(
    title='Evolução do Preço das Ações ao Longo do Tempo',
    legend_title_text='TICKER',
    xaxis_title='Data',
    yaxis_title='Preço',
    height=1000
)

figura.show()

# ANÁLISESSSSSS @@@@@@@@@@@@@@@@@

In [284]:
df

,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,...,MÁXIMO DROW DOWN,MÁXIMO DROW DOWN bruto,VOLATILIDADE,RENTABILIDADE,GANHO BRUTO R$,ACUM_GANHO BRUTO R$,ACUM_VOLATILIDADE,ACUM_MÉDIA,ACUM_RENTABILIDADE,DROW_DOWN
PAPEL,,,,,,,,,,,,,,,,,,,,,
ABEV3,Jan-2023,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,NaN,...,-0.019766,-0.27,0.043101,0.128111,1.75,0.00,NaN,13.660000,NaN,0.000000
ABEV3,Feb-2023,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,-0.019766,...,-0.019766,-0.27,0.043101,0.128111,1.75,-0.27,NaN,13.525000,-0.019766,-0.019766
ABEV3,Mar-2023,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,0.070202,...,-0.019766,-0.27,0.043101,0.128111,1.75,0.67,0.063617,13.793333,0.049048,0.000000
ABEV3,Apr-2023,AMBEV S/A,14.16,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0061,0.0092,5.0001,-0.011863,...,-0.019766,-0.27,0.043101,0.128111,1.75,0.50,0.049818,13.885000,0.036603,-0.011863
ABEV3,May-2023,AMBEV S/A,14.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0023,0.0112,5.0953,0.017655,...,-0.019766,-0.27,0.043101,0.128111,1.75,0.75,0.040747,13.990000,0.054905,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YDUQ3,Feb-2023,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0084,0.0092,5.2072,-0.317522,...,-0.317522,-3.28,0.399190,0.918683,9.49,-3.28,NaN,8.690000,-0.317522,-0.317522
YDUQ3,Mar-2023,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0071,0.0117,5.0798,0.000000,...,-0.317522,-3.28,0.399190,0.918683,9.49,-3.28,0.224522,8.143333,-0.317522,-0.317522
YDUQ3,Apr-2023,YDUQS PART,8.12,Consumo Cíclico,Diversos,Serviços Educacionais,0.0061,0.0092,5.0001,0.151773,...,-0.317522,-3.28,0.399190,0.918683,9.49,-2.21,0.239476,8.137500,-0.213940,-0.213940


In [ ]:
import streamlit as st 

# @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

In [139]:
# df.rename(columns={'MÁXIMO DROW DOWN': 'MAX DD (%)', 'MÁXIMO DROW DOWN bruto': 'MAX DD (R$)'}, inplace=True)

In [253]:
df.columns

Index(['DATA', 'NOME', 'PREÇO (R$)', 'SETOR ECONÔMICO', 'SUBSETOR', 'SEGMENTO',
       'IPCA', 'SELIC', 'USD', 'RETORNOS_SIMPLES', 'RETORNOS_LOGARITIMOS',
       'RETORNOS_ACUMULADOS', 'SHARPE', 'MÉDIA', 'MÁXIMO DROW DOWN',
       'MÁXIMO DROW DOWN bruto', 'VOLATILIDADE', 'RENTABILIDADE',
       'GANHO BRUTO R$', 'ACUM_GANHO BRUTO R$', 'ACUM_VOLATILIDADE',
       'ACUM_MÉDIA', 'ACUM_RENTABILIDADE', 'DROW_DOWN'],
      dtype='object')

In [257]:
df.reset_index(inplace=True)

,PAPEL,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,...,MÁXIMO DROW DOWN,MÁXIMO DROW DOWN bruto,VOLATILIDADE,RENTABILIDADE,GANHO BRUTO R$,ACUM_GANHO BRUTO R$,ACUM_VOLATILIDADE,ACUM_MÉDIA,ACUM_RENTABILIDADE,DROW_DOWN
0,ABEV3,Jan-2023,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,...,-0.019766,-0.27,0.043101,0.128111,1.75,0.00,NaN,13.660000,NaN,0.000000
1,ABEV3,Feb-2023,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,...,-0.019766,-0.27,0.043101,0.128111,1.75,-0.27,NaN,13.525000,-0.019766,-0.019766
2,ABEV3,Mar-2023,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,...,-0.019766,-0.27,0.043101,0.128111,1.75,0.67,0.063617,13.793333,0.049048,0.000000
3,ABEV3,Apr-2023,AMBEV S/A,14.16,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0061,0.0092,5.0001,...,-0.019766,-0.27,0.043101,0.128111,1.75,0.50,0.049818,13.885000,0.036603,-0.011863
4,ABEV3,May-2023,AMBEV S/A,14.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0023,0.0112,5.0953,...,-0.019766,-0.27,0.043101,0.128111,1.75,0.75,0.040747,13.990000,0.054905,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,YDUQ3,Feb-2023,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0084,0.0092,5.2072,...,-0.317522,-3.28,0.399190,0.918683,9.49,-3.28,NaN,8.690000,-0.317522,-0.317522
524,YDUQ3,Mar-2023,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0071,0.0117,5.0798,...,-0.317522,-3.28,0.399190,0.918683,9.49,-3.28,0.224522,8.143333,-0.317522,-0.317522
525,YDUQ3,Apr-2023,YDUQS PART,8.12,Consumo Cíclico,Diversos,Serviços Educacionais,0.0061,0.0092,5.0001,...,-0.317522,-3.28,0.399190,0.918683,9.49,-2.21,0.239476,8.137500,-0.213940,-0.213940
526,YDUQ3,May-2023,YDUQS PART,14.07,Consumo Cíclico,Diversos,Serviços Educacionais,0.0023,0.0112,5.0953,...,-0.317522,-3.28,0.399190,0.918683,9.49,3.74,0.439854,9.324000,0.362052,0.000000


In [294]:
df['DATA']

PAPEL
ABEV3   2023-01-31
ABEV3   2023-02-28
ABEV3   2023-03-31
ABEV3   2023-04-30
ABEV3   2023-05-31
           ...    
YDUQ3   2023-02-28
YDUQ3   2023-03-31
YDUQ3   2023-04-30
YDUQ3   2023-05-31
YDUQ3   2023-06-30
Name: DATA, Length: 528, dtype: datetime64[ns]

In [298]:
df['ACUM_GANHO BRUTO R$'] = df['PREÇO (R$)'].groupby(df.index).transform(lambda x : x - x.iloc[0])
df['ACUM_VOLATILIDADE'] = df[['RETORNOS_SIMPLES']].groupby(df.index).expanding(min_periods=0).std().values# * 100
df['ACUM_MÉDIA'] = df[['PREÇO (R$)']].groupby(df.index).expanding(min_periods=1).mean().values
df['ACUM_RENTABILIDADE'] = df[['RETORNOS_SIMPLES']].groupby(df.index).transform(lambda x : ((x + 1).cumprod() - 1))# * 100 
df['DROW_DOWN'] = drowdowns_neg.values# * 100
# df['DATA'] = df['DATA'].dt.strftime('%b-%Y')
df['DATA'] = pd.to_datetime(df['DATA']).dt.date
df.fillna(0)


,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,...,MÁXIMO DROW DOWN,MÁXIMO DROW DOWN bruto,VOLATILIDADE,RENTABILIDADE,GANHO BRUTO R$,ACUM_GANHO BRUTO R$,ACUM_VOLATILIDADE,ACUM_MÉDIA,ACUM_RENTABILIDADE,DROW_DOWN
PAPEL,,,,,,,,,,,,,,,,,,,,,
ABEV3,2023-01-31,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,0.000000,...,-0.019766,-0.27,0.043101,0.128111,1.75,0.00,0.000000,13.660000,0.000000,0.000000
ABEV3,2023-02-28,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,-0.019766,...,-0.019766,-0.27,0.043101,0.128111,1.75,-0.27,0.000000,13.525000,-0.019766,-0.019766
ABEV3,2023-03-31,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,0.070202,...,-0.019766,-0.27,0.043101,0.128111,1.75,0.67,0.063617,13.793333,0.049048,0.000000
ABEV3,2023-04-30,AMBEV S/A,14.16,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0061,0.0092,5.0001,-0.011863,...,-0.019766,-0.27,0.043101,0.128111,1.75,0.50,0.049818,13.885000,0.036603,-0.011863
ABEV3,2023-05-31,AMBEV S/A,14.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0023,0.0112,5.0953,0.017655,...,-0.019766,-0.27,0.043101,0.128111,1.75,0.75,0.040747,13.990000,0.054905,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YDUQ3,2023-02-28,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0084,0.0092,5.2072,-0.317522,...,-0.317522,-3.28,0.399190,0.918683,9.49,-3.28,0.000000,8.690000,-0.317522,-0.317522
YDUQ3,2023-03-31,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0071,0.0117,5.0798,0.000000,...,-0.317522,-3.28,0.399190,0.918683,9.49,-3.28,0.224522,8.143333,-0.317522,-0.317522
YDUQ3,2023-04-30,YDUQS PART,8.12,Consumo Cíclico,Diversos,Serviços Educacionais,0.0061,0.0092,5.0001,0.151773,...,-0.317522,-3.28,0.399190,0.918683,9.49,-2.21,0.239476,8.137500,-0.213940,-0.213940


In [227]:
df.head(15)

,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,...,MÁXIMO DROW DOWN,MÁXIMO DROW DOWN bruto,VOLATILIDADE,RENTABILIDADE,GANHO BRUTO R$,ACUM_GANHO BRUTO R$,ACUM_VOLATILIDADE,ACUM_MÉDIA,ACUM_RENTABILIDADE,ACUM_DROW_DOWN
PAPEL,,,,,,,,,,,,,,,,,,,,,
ABEV3,2023-01-31,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,NaN,...,-0.019766,-0.27,0.043101,0.128111,1.75,0.00,NaN,13.660000,NaN,0.000000
ABEV3,2023-02-28,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,-0.019766,...,-0.019766,-0.27,0.043101,0.128111,1.75,-0.27,NaN,13.525000,-0.019766,-0.019766
ABEV3,2023-03-31,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,0.070202,...,-0.019766,-0.27,0.043101,0.128111,1.75,0.67,0.063617,13.793333,0.049048,0.000000
ABEV3,2023-04-30,AMBEV S/A,14.16,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0061,0.0092,5.0001,-0.011863,...,-0.019766,-0.27,0.043101,0.128111,1.75,0.50,0.049818,13.885000,0.036603,-0.011863
ABEV3,2023-05-31,AMBEV S/A,14.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0023,0.0112,5.0953,0.017655,...,-0.019766,-0.27,0.043101,0.128111,1.75,0.75,0.040747,13.990000,0.054905,0.000000
ABEV3,2023-06-30,AMBEV S/A,15.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,-0.0008,0.0107,4.8186,0.069396,...,-0.019766,-0.27,0.043101,0.128111,1.75,1.75,0.043101,14.226667,0.128111,0.000000
AESB3,2023-01-31,AES BRASIL,10.10,Utilidade Pública,Energia Elétrica,Energia Elétrica,0.0053,0.0112,5.0987,NaN,...,-0.017822,-0.18,0.065634,0.212871,2.15,0.00,NaN,10.100000,NaN,0.000000
AESB3,2023-02-28,AES BRASIL,9.92,Utilidade Pública,Energia Elétrica,Energia Elétrica,0.0084,0.0092,5.2072,-0.017822,...,-0.017822,-0.18,0.065634,0.212871,2.15,-0.18,NaN,10.010000,-0.017822,-0.017822
AESB3,2023-03-31,AES BRASIL,9.94,Utilidade Pública,Energia Elétrica,Energia Elétrica,0.0071,0.0117,5.0798,0.002016,...,-0.017822,-0.18,0.065634,0.212871,2.15,-0.16,0.014028,9.986667,-0.015842,-0.015842


In [297]:
df.to_excel('df_completo2.xlsx')
# df.to_excel('df_completo.xlsx')

In [156]:
# df = pd.read_excel('df_completo.xlsx', index_col='DATA')
# df.to_excel('df_completo.xlsx')

In [235]:
from plotly.subplots import make_subplots   
from plotly import graph_objects as go 

In [236]:
df.head(3)

,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,...,MÁXIMO DROW DOWN,MÁXIMO DROW DOWN bruto,VOLATILIDADE,RENTABILIDADE,GANHO BRUTO R$,ACUM_GANHO BRUTO R$,ACUM_VOLATILIDADE,ACUM_MÉDIA,ACUM_RENTABILIDADE,DROW_DOWN
PAPEL,,,,,,,,,,,,,,,,,,,,,
ABEV3,2023-01-31,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,NaN,...,-0.019766,-0.27,0.043101,0.128111,1.75,0.00,NaN,13.660000,NaN,0.000000
ABEV3,2023-02-28,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,-0.019766,...,-0.019766,-0.27,0.043101,0.128111,1.75,-0.27,NaN,13.525000,-0.019766,-0.019766
ABEV3,2023-03-31,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,0.070202,...,-0.019766,-0.27,0.043101,0.128111,1.75,0.67,0.063617,13.793333,0.049048,0.000000


In [ ]:
figura = make_subplots(rows = 1, cols=1, subplot_titles=['RENTABILIDADE MENSAL'])#, 'RENTABILIDADE ACUMULADA', 'GANHO BRUTO ACUMULADO', 'DROW DOWN'])

figura.add_trace(go.Scatter(x=df['DATA'].unique(), y=rent_max['RETORNOS_SIMPLES'], 
                            name=rent_max['NOME'].unique()), row=1, col=1)

In [ ]:
figura = make_subplots(rows = 1, cols=1, subplot_titles=['', '', 'GANHO BRUTO ACUMULADO', 'DROW DOWN'])

figura.add_trace(go.Scatter(x=df['DATA'], y=df['RETORNOS_SIMPLES'].loc[df.index == stock_compare_choose], name=stock_compare_choose), row=1, col=1)
figura.add_trace(go.Scatter(x=df['DATA'], y=rent_max, 
                            name=rent_max.index.unique()), row=1, col=1)
# figura.add_trace(go.Scatter(x=df['DATA'], y=df['Informacao2'], name='Melhor Ação'), row=1, col=2)
# figura.add_trace(go.Bar(x=df['DATA'], y=df['Informacao3'], name='Ação'), row=2, col=1)
# figura.add_trace(go.Bar(x=df['DATA'], y=df['Informacao4'], name='Melhor Ação'), row=2, col=2)

# @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

In [262]:
df.columns

Index(['DATA', 'NOME', 'PREÇO (R$)', 'SETOR ECONÔMICO', 'SUBSETOR', 'SEGMENTO',
       'IPCA', 'SELIC', 'USD', 'RETORNOS_SIMPLES', 'RETORNOS_LOGARITIMOS',
       'RETORNOS_ACUMULADOS', 'SHARPE', 'MÉDIA', 'MÁXIMO DROW DOWN',
       'MÁXIMO DROW DOWN bruto', 'VOLATILIDADE', 'RENTABILIDADE',
       'GANHO BRUTO R$', 'ACUM_GANHO BRUTO R$', 'ACUM_VOLATILIDADE',
       'ACUM_MÉDIA', 'ACUM_RENTABILIDADE', 'DROW_DOWN'],
      dtype='object')

In [264]:
df.head(1)

,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,...,MÁXIMO DROW DOWN,MÁXIMO DROW DOWN bruto,VOLATILIDADE,RENTABILIDADE,GANHO BRUTO R$,ACUM_GANHO BRUTO R$,ACUM_VOLATILIDADE,ACUM_MÉDIA,ACUM_RENTABILIDADE,DROW_DOWN
PAPEL,,,,,,,,,,,,,,,,,,,,,
ABEV3,Jan-2023,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,NaN,...,-0.019766,-0.27,0.043101,0.128111,1.75,0.0,NaN,13.66,NaN,0.0


## Por SETOR ECONOMICO

In [ ]:
df.groupby('SETOR ECONÔMICO')['PREÇO (R$)'].mean().sort_values(ascending=False)

In [ ]:
df.groupby('SETOR ECONÔMICO')['SHARPE'].mean().sort_values(ascending=False)


In [ ]:
df.groupby('SETOR ECONÔMICO')['VOLATILIDADE'].mean().sort_values(ascending=True)


In [ ]:
df.groupby('SETOR ECONÔMICO')['VOLATILIDADE'].mean().sort_values(ascending=True)


In [276]:
participacao = df['SEGMENTO'].value_counts(normalize=True).sort_values(ascending=False)
participacao[participacao.index == 'Siderurgia'].values[0]

0.03409090909090909

In [282]:
participacao = df['SEGMENTO'].value_counts(normalize=False).sort_values(ascending=False)
participacao[participacao.index == 'Siderurgia']

88.0

## Por SUB SETOR 

In [ ]:
df.groupby('SUBSETOR')['PREÇO (R$)'].mean().sort_values(ascending=False)

# TESTES GRAFICO DE BARRAS

In [66]:
tickers = ['ABEV3', 'YDUQ3','AZUL4']
best_tickers = df.loc[: , ['MÉDIA' ,'SHARPE', 'MÁXIMO DROW DOWN', 'MÁXIMO DROW DOWN bruto', 'VOLATILIDADE', 'RENTABILIDADE', 'GANHO BRUTO R$']].idxmax().values

max_values = df.loc[: , ['MÉDIA' ,'SHARPE', 'MÁXIMO DROW DOWN', 'MÁXIMO DROW DOWN bruto', 'VOLATILIDADE', 'RENTABILIDADE', 'GANHO BRUTO R$']].max().values
info = df.loc[: , ['MÉDIA' ,'SHARPE', 'MÁXIMO DROW DOWN', 'MÁXIMO DROW DOWN bruto', 'VOLATILIDADE', 'RENTABILIDADE', 'GANHO BRUTO R$']].max().index.values


In [67]:
best_tickers, info, max_values

(array(['VALE3', 'POMO4', 'POMO4', 'POMO4', 'AZUL4', 'YDUQ3', 'AZUL4'],
       dtype=object),
 array(['MÉDIA', 'SHARPE', 'MÁXIMO DROW DOWN', 'MÁXIMO DROW DOWN bruto',
        'VOLATILIDADE', 'RENTABILIDADE', 'GANHO BRUTO R$'], dtype=object),
 array([76.75833333,  6.3193426 ,  0.        ,  0.        ,  0.45179676,
         0.91868345, 10.01      ]))

In [68]:
tudu_juntu = zip(best_tickers, info, max_values)
lista_tudu_junto = list(tudu_juntu)

In [69]:
ltj = lista_tudu_junto
ltj

[('VALE3', 'MÉDIA', 76.75833333333334),
 ('POMO4', 'SHARPE', 6.3193426033136655),
 ('POMO4', 'MÁXIMO DROW DOWN', 0.0),
 ('POMO4', 'MÁXIMO DROW DOWN bruto', 0.0),
 ('AZUL4', 'VOLATILIDADE', 0.45179675954297854),
 ('YDUQ3', 'RENTABILIDADE', 0.9186834462729913),
 ('AZUL4', 'GANHO BRUTO R$', 10.01)]

In [158]:
lista_acoes = ['AESB3', 'ABEV3']

In [171]:
df.loc[df.index == 'ABEV3', info]

,MÉDIA,SHARPE,MÁXIMO DROW DOWN,MÁXIMO DROW DOWN bruto,VOLATILIDADE,RENTABILIDADE,GANHO BRUTO R$
PAPEL,,,,,,,
ABEV3,14.226667,1.92827,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,14.226667,1.92827,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,14.226667,1.92827,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,14.226667,1.92827,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,14.226667,1.92827,-0.019766,-0.27,0.043101,0.128111,1.75
ABEV3,14.226667,1.92827,-0.019766,-0.27,0.043101,0.128111,1.75


In [180]:
figura = go.Figure()

# Adiciona barras para cada indicador
for acao in lista_acoes:
    figura.add_trace(go.Bar(name=acao, x=info, 
                            y=round(df.loc[df.index == acao, info],2),
                            marker_color='blue',)
                            )
    
    
# Layout do gráfico
figura.update_layout(barmode='group', title='Valores Máximos por Indicador e Ação',
                     xaxis_title='Informações', yaxis_title='Valor',)
figura.show()

In [152]:
info = ['MÉDIA', 'SHARPE', 'MÁXIMO DROW DOWN', 'MÁXIMO DROW DOWN bruto', 'VOLATILIDADE', 'RENTABILIDADE', 'GANHO BRUTO R$']
best_tickers = ['VALE3', 'POMO4', 'POMO4', 'POMO4', 'AZUL4', 'YDUQ3', 'AZUL4']
max_values = [76.76, 6.32, 0.0, 0.0, 0.45, 0.92, 10.01]

# Criação do gráfico de barras empilhadas
figura = go.Figure()

# Adiciona barras para cada indicador
for i, informacao in enumerate(info):
    # Use best_tickers[i] como o valor para o eixo x
    text_values = [f"{informacao}: {value}" for value in max_values]
    figura.add_trace(go.Bar(name=best_tickers[i], x=[informacao], y=[max_values[i]], marker_color='blue'))

# Layout do gráfico
# figura.update_traces(hoverinfo='text+y+x')
figura.update_layout(barmode='group', title='Valores Máximos por Indicador e Ação',
                     xaxis_title='Indicadores', yaxis_title='Valor')

# Exibe o gráfico
figura.show()

In [184]:
tickers = ['ABEV3', 'YDUQ3','AZUL4']
best_tickers = df.loc[: , ['MÉDIA' ,'SHARPE', 'MÁXIMO DROW DOWN', 'MÁXIMO DROW DOWN bruto', 'VOLATILIDADE', 'RENTABILIDADE', 'GANHO BRUTO R$']].idxmax().values

max_values = df.loc[: , ['MÉDIA' ,'SHARPE', 'MÁXIMO DROW DOWN', 'MÁXIMO DROW DOWN bruto', 'VOLATILIDADE', 'RENTABILIDADE', 'GANHO BRUTO R$']].max().values
info = df.loc[: , ['MÉDIA' ,'SHARPE', 'MÁXIMO DROW DOWN', 'MÁXIMO DROW DOWN bruto', 'VOLATILIDADE', 'RENTABILIDADE', 'GANHO BRUTO R$']].max().index.values

acoes_selecionadas = tickers
acoes_selecionadas
info
figura = go.Figure()

# Adiciona barras para cada indicador
for acao in acoes_selecionadas:
    figura.add_trace(go.Bar(name=acao, x=info, 
                            y=round(df.loc[df.index == acao, info],2),
                            marker_color='blue',)
                            )
    
    
# Layout do gráfico
figura.update_layout(barmode='group', title='Valores Máximos por Indicador e Ação',
                    xaxis_title='Informações', yaxis_title='Valor',)
# st.plotly_chart(figura)
figura.show()

In [ ]:
todos_tickers = df.index
todos_tickers = todos_tickers.value_counts().index
todos_tickers

In [210]:
df_completo

,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,RETORNOS_LOGARITIMOS,RETORNOS_ACUMULADOS
PAPEL,,,,,,,,,,,,
ABEV3,2023-01-31,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,NaN,NaN,NaN
ABEV3,2023-02-28,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,-0.019766,-0.019964,-0.019766
ABEV3,2023-03-31,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,0.070202,0.067847,0.049048
ABEV3,2023-04-30,AMBEV S/A,14.16,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0061,0.0092,5.0001,-0.011863,-0.011934,0.036603
ABEV3,2023-05-31,AMBEV S/A,14.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0023,0.0112,5.0953,0.017655,0.017501,0.054905
...,...,...,...,...,...,...,...,...,...,...,...,...
YDUQ3,2023-02-28,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0084,0.0092,5.2072,-0.317522,-0.382025,-0.317522
YDUQ3,2023-03-31,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0071,0.0117,5.0798,0.000000,0.000000,-0.317522
YDUQ3,2023-04-30,YDUQS PART,8.12,Consumo Cíclico,Diversos,Serviços Educacionais,0.0061,0.0092,5.0001,0.151773,0.141303,-0.213940


In [223]:
import plotly.graph_objects as go
from plotly import data

df_teste = data.medals_wide()

fig = go.Figure()

# Iterar sobre as colunas do DataFrame
for col in ['gold', 'silver', 'bronze']:
    fig.add_trace(go.Bar(x=df_teste.nation, y=df_teste[col], name=col))

fig.update_layout(barcornerradius=15)
fig.show()

In [285]:
df = t7.copy(deep=True)
df.reset_index(inplace=True)
df

,PAPEL,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,RETORNOS_LOGARITIMOS,RETORNOS_ACUMULADOS,SHARPE,MÉDIA,MÁXIMO DROW DOWN,MÁXIMO DROW DOWN bruto,VOLATILIDADE,RENTABILIDADE,GANHO BRUTO R$
0,ABEV3,2023-01-31,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,NaN,NaN,NaN,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
1,ABEV3,2023-02-28,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,-0.019766,-0.019964,-0.019766,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
2,ABEV3,2023-03-31,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,0.070202,0.067847,0.049048,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
3,ABEV3,2023-04-30,AMBEV S/A,14.16,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0061,0.0092,5.0001,-0.011863,-0.011934,0.036603,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
4,ABEV3,2023-05-31,AMBEV S/A,14.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0023,0.0112,5.0953,0.017655,0.017501,0.054905,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,YDUQ3,2023-02-28,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0084,0.0092,5.2072,-0.317522,-0.382025,-0.317522,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49
524,YDUQ3,2023-03-31,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0071,0.0117,5.0798,0.000000,0.000000,-0.317522,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49
525,YDUQ3,2023-04-30,YDUQS PART,8.12,Consumo Cíclico,Diversos,Serviços Educacionais,0.0061,0.0092,5.0001,0.151773,0.141303,-0.213940,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49
526,YDUQ3,2023-05-31,YDUQS PART,14.07,Consumo Cíclico,Diversos,Serviços Educacionais,0.0023,0.0112,5.0953,0.732759,0.549715,0.362052,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49


In [264]:
import time

In [322]:
info

array(['MÉDIA', 'SHARPE', 'MÁXIMO DROW DOWN', 'MÁXIMO DROW DOWN bruto',
       'VOLATILIDADE', 'RENTABILIDADE', 'GANHO BRUTO R$'], dtype=object)

In [344]:
df

,PAPEL,DATA,NOME,PREÇO (R$),SETOR ECONÔMICO,SUBSETOR,SEGMENTO,IPCA,SELIC,USD,RETORNOS_SIMPLES,RETORNOS_LOGARITIMOS,RETORNOS_ACUMULADOS,SHARPE,MÉDIA,MÁXIMO DROW DOWN,MÁXIMO DROW DOWN bruto,VOLATILIDADE,RENTABILIDADE,GANHO BRUTO R$
0,ABEV3,2023-01-31,AMBEV S/A,13.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0053,0.0112,5.0987,NaN,NaN,NaN,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
1,ABEV3,2023-02-28,AMBEV S/A,13.39,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0084,0.0092,5.2072,-0.019766,-0.019964,-0.019766,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
2,ABEV3,2023-03-31,AMBEV S/A,14.33,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0071,0.0117,5.0798,0.070202,0.067847,0.049048,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
3,ABEV3,2023-04-30,AMBEV S/A,14.16,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0061,0.0092,5.0001,-0.011863,-0.011934,0.036603,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
4,ABEV3,2023-05-31,AMBEV S/A,14.41,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,0.0023,0.0112,5.0953,0.017655,0.017501,0.054905,1.928270,14.226667,-0.019766,-0.27,0.043101,0.128111,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,YDUQ3,2023-02-28,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0084,0.0092,5.2072,-0.317522,-0.382025,-0.317522,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49
524,YDUQ3,2023-03-31,YDUQS PART,7.05,Consumo Cíclico,Diversos,Serviços Educacionais,0.0071,0.0117,5.0798,0.000000,0.000000,-0.317522,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49
525,YDUQ3,2023-04-30,YDUQS PART,8.12,Consumo Cíclico,Diversos,Serviços Educacionais,0.0061,0.0092,5.0001,0.151773,0.141303,-0.213940,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49
526,YDUQ3,2023-05-31,YDUQS PART,14.07,Consumo Cíclico,Diversos,Serviços Educacionais,0.0023,0.0112,5.0953,0.732759,0.549715,0.362052,2.188639,11.073333,-0.317522,-3.28,0.399190,0.918683,9.49


In [342]:
fig = go.Figure()

# Iterar sobre as colunas do DataFrame
for col in ['gold', 'silver', 'bronze']:
    fig.add_trace(go.Bar(x=df_teste.nation, y=df_teste[col], name=col))

fig.update_layout(barcornerradius=15)
fig.show()

In [288]:
df_teste = data.medals_wide()

fig = make_subplots(rows=1, cols=len(acoes_selecionadas), shared_yaxes=True)


# Iterar sobre as colunas do DataFrame
for i, col in enumerate(df_teste.columns[1:], start=1):
    fig.add_trace(
        go.Bar(x=df_teste.nation, y=df_teste[col], name=col, ),
        row=1, col=i
    )

fig.show()